In [ ]:
# License: BSD
# Author: Sasank Chilamkurthy

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

os.environ["CUDA_VISIBLE_DEVICES"]="5"

plt.ion()   # interactive mode

# Data augmentation and normalization for training
# Just normalization for test
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        # transforms.Normalize([0.7206, 0.7204, 0.7651], [0.2305, 0.2384, 0.1706])
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.ToTensor(),
        # transforms.Normalize([0.7206, 0.7204, 0.7651], [0.2305, 0.2384, 0.1706])
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

data_dir = "/export/gaon1/data/jteneggi/data/malaria/trophozoite"
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'test']:
            model.train()  # Set model to training mode
            model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, 2)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.2)

model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=25)

torch.save(model_ft.state_dict(), "ResNet18")

Epoch 0/24
----------
train Loss: 0.6555 Acc: 0.6257
test Loss: 0.6698 Acc: 0.5677

Epoch 1/24
----------
train Loss: 0.6075 Acc: 0.6775
test Loss: 0.5905 Acc: 0.6376

Epoch 2/24
----------
train Loss: 0.6118 Acc: 0.6099
test Loss: 0.6928 Acc: 0.5066

Epoch 3/24
----------
train Loss: 0.6198 Acc: 0.6249
test Loss: 0.4270 Acc: 0.8122

Epoch 4/24
----------
train Loss: 0.5016 Acc: 0.7485
test Loss: 0.3626 Acc: 0.8384

Epoch 5/24
----------
train Loss: 0.5229 Acc: 0.7419
test Loss: 0.4112 Acc: 0.8079

Epoch 6/24
----------
train Loss: 0.4223 Acc: 0.7987
test Loss: 0.4871 Acc: 0.7773

Epoch 7/24
----------
train Loss: 0.4184 Acc: 0.8070
test Loss: 0.3508 Acc: 0.8603

Epoch 8/24
----------
train Loss: 0.4459 Acc: 0.7886
test Loss: 0.3382 Acc: 0.8428

Epoch 9/24
----------
train Loss: 0.4182 Acc: 0.7962
test Loss: 0.4286 Acc: 0.7729

Epoch 10/24
----------
train Loss: 0.3487 Acc: 0.8296
test Loss: 0.3243 Acc: 0.8559

Epoch 11/24
----------
train Loss: 0.3300 Acc: 0.8538
test Loss: 0.3152 Acc

In [ ]:
x = np.arange(25)
fig = plt.figure()
ax = fig.subplots(1, 1)

ax.plot(x, train_loss, label="loss")
ax.plot(x, train_acc, label="accuracy")
ax.set_xlabel("Epoch")
ax.set_title("Train")
ax.legend()
plt.savefig("convergence.jpg")